## Import library

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime

import yfinance as yf
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import sambo
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy

from backtesting.test import SMA, GOOG

# backtesting.set_bokeh_output(notebook=True)
            
import itertools 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14340\1650128870.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\Binance Tradebot\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Import utils

In [2]:
from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *

## Plotting sample

In [39]:
# now = datetime.today().strftime('%Y-%m-%d')

loader = DataLoader(ticker='INTC', start='2010-01-01', end='2020-12-31', freq='1d', test_size=0.2)
loader.run()
# test     

[*********************100%***********************]  1 of 1 completed


## RSI strategy evaluation 

In [41]:
def RSI(arr, n: int = 10, k: int = 10) -> pd.Series:
    """
    Returns `n`-period Relative Strength Index (RSI) of array `arr`.
    """
    arr = pd.Series(arr)  # Ensure arr is a Pandas Series
    delta = np.diff(arr, prepend=arr[0])  # Compute differences manually

    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(n).mean()
    avg_loss = pd.Series(loss).rolling(n).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    rsi_ma = rsi.rolling(k).mean()

    return rsi, rsi_ma

def BBW(arr, n: int = 10) -> pd.Series:
    """
    Returns Bollinger Band Width (BBW) of array `arr` over `n` periods.
    `k` is the smoothing period for signal generation.
    """
    arr = pd.Series(arr)
    
    sma = arr.rolling(n).mean()  # Middle Band
    std = arr.rolling(n).std()   # Standard Deviation
    
    upper_band = sma + 2 * std
    lower_band = sma - 2 * std

    bbw = (upper_band - lower_band) / sma  # Normalize by SMA
        
    return bbw

class RSICross(Strategy):

    short_duration = 10  # Default values, can be overridden
    long_duration = 50

    id = 0
    take_profit_ratio = 1.3
    stop_loss_ratio = 0.9

    stop_loss_duration = 5

    lookback = 3
    atr_multiplier = 1

    def init(self):
        price = self.data.Close
        self.ma1 = self.I(RSI, price, plot=True, overlay=False)[0]
        self.ma2 = self.I(RSI, price, plot=False, overlay=False)[1]
        self.bbw = self.I(BBW, price, plot=True, overlay=False)
        # self.atr = self.I(ATR, price, plot=True, overlay=False)
        self.ma200 = self.I(EMA, price, 20)

        self.atr = self.I(ATR, self.data)
        self.previous_low = self.I(previous_low, self.data.Low, self.stop_loss_duration)
        self.previous_high = self.I(previous_high, self.data.High, self.stop_loss_duration)

    def next(self):
        entry_price = self.data.Close[-1]

        long_stop_loss = min(self.previous_low[-1], entry_price - self.atr_multiplier*self.atr[-1])

        for trade in self.trades:
            trade.sl = long_stop_loss


        past_max = max(self.bbw[-self.lookback-1:-1])  # Maximum in last 5 periods
        current_bbw = self.bbw[-1]

        count_green = sum(self.data.Color[-2:])

        if crossover(self.ma1, self.ma2) and current_bbw > past_max and count_green >= 2:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long {self.id}')
            self.id += 1
        
        if self.ma1[-1] > self.ma2[-1] and self.ma1[-2] > self.ma2[-2] and count_green >= 2:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long {self.id}')
            self.id += 1

        # elif crossover(self.ma2, self.ma1):
        #     for trade in self.trades:
        #         if trade.tag ==f'Long {self.id-1}':
        #             trade.close()

strategy = RSICross
# strategy = BollingerBound
bt = BackTrader(data=loader.data)

# params = {
#     'short_duration': 5,
#     'long_duration': 10
# }
params = {'take_profit_ratio': 1.3}
bt.evaluate(data=bt.test_data, strategy=strategy, params=params, order_size=0.9999, plot=True)
# bt.trades.head()

Start                     2018-10-18 00:00:00
End                       2020-12-30 00:00:00
Duration                    804 days 00:00:00
Exposure Time [%]                    54.33213
Equity Final [$]                  16139.22141
Equity Peak [$]                    18740.4078
Commissions [$]                     797.50579
Return [%]                           61.39221
Buy & Hold Return [%]                  8.0579
Return (Ann.) [%]                    24.32553
Volatility (Ann.) [%]                33.50816
CAGR [%]                             16.18692
Sharpe Ratio                          0.72596
Sortino Ratio                          1.3287
Calmar Ratio                          1.15672
Max. Drawdown [%]                   -21.02973
Avg. Drawdown [%]                    -4.31204
Max. Drawdown Duration      208 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   16
Win Rate [%]                            68.75
Best Trade [%]                    

## MACD strategy evaluation

In [27]:
class EMACross_test(Strategy):

    short_duration = 5  # Default values, can be overridden
    long_duration = 10

    id = 0
    take_profit_ratio = 1.3
    stop_loss_ratio = 0.9

    stop_loss_duration = 5

    lookback = 3
    atr_multiplier = 1

    def init(self):
        price = self.data.Close
        self.ma1 = self.I(EMA, price, self.short_duration, plot=True, overlay=True)
        self.ma2 = self.I(EMA, price, self.long_duration, plot=True, overlay=True)
        self.bbw = self.I(BBW, price, plot=False, overlay=False)
        # self.atr = self.I(ATR, price, plot=True, overlay=False)
        self.ma200 = self.I(EMA, price, 100)

        self.atr = self.I(ATR, self.data)
        self.previous_low = self.I(previous_low, self.data.Low, self.stop_loss_duration, plot = False)
        self.previous_high = self.I(previous_high, self.data.High, self.stop_loss_duration, plot = False)

    def next(self):
        entry_price = self.data.Close[-1]

        long_stop_loss = min(self.previous_low[-1], entry_price - self.atr_multiplier*self.atr[-1])
        short_stop_loss = max(self.previous_high[-1], entry_price + self.atr_multiplier*self.atr[-1])

        for trade in self.trades:
            if trade.tag == "Long":
                trade.sl = long_stop_loss
            elif trade.tag == "Short":
                trade.sl = short_stop_loss


        past_max = max(self.bbw[-self.lookback-1:-1])  # Maximum in last 5 periods
        current_bbw = self.bbw[-1]

        count_green = sum(self.data.Color[-2:])

        if crossover(self.ma1, self.ma2) and count_green >= 2 and current_bbw >= past_max:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long')
            
            for trade in self.trades:
                if trade.tag == 'Short':
                    trade.close()

        if crossover(self.ma2, self.ma1) and count_green == 0 and current_bbw >= past_max:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Short')
            
            for trade in self.trades:
                if trade.tag == 'Long':
                    trade.close()


        # elif crossover(self.ma2, self.ma1):
        #     for trade in self.trades:
        #         if trade.tag ==f'Long {self.id-1}':
        #             trade.close()

strategy = EMACross_test
# strategy = BollingerBound
bt = BackTrader(data=loader.data)

# params = {
#     'short_duration': 5,
#     'long_duration': 10
# }
params = {'take_profit_ratio': 1.3}
bt.evaluate(data=bt.train_data, strategy=strategy, params=params, order_size=0.9999, plot=True)
# bt.trades.head()

Start                     2010-01-04 00:00:00
End                       2018-10-17 00:00:00
Duration                   3208 days 00:00:00
Exposure Time [%]                    11.83379
Equity Final [$]                  13677.86476
Equity Peak [$]                   13783.78617
Commissions [$]                    1429.02547
Return [%]                           36.77865
Buy & Hold Return [%]               783.46244
Return (Ann.) [%]                     3.62911
Volatility (Ann.) [%]                 7.62251
CAGR [%]                              2.49076
Sharpe Ratio                           0.4761
Sortino Ratio                         0.85533
Calmar Ratio                          0.24324
Max. Drawdown [%]                   -14.92001
Avg. Drawdown [%]                    -2.54084
Max. Drawdown Duration     1596 days 00:00:00
Avg. Drawdown Duration      179 days 00:00:00
# Trades                                   32
Win Rate [%]                           59.375
Best Trade [%]                    

## Cross-validation sample

In [43]:
# grid_search = {
#     'short_duration': range(2, 4),
#     'long_duration': range(5, 11)
# }
grid_search={'take_profit_ratio': [1.3], 'stop_loss_ratio': [0.9]}

bt.cross_val(strategy=strategy, train_size=240, test_size=240, step_size=240, order_size=0.999, commission=0.002, grid=grid_search)

{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
10117.372469282258
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
7989.234014483429
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
10238.619861494932
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
12615.28934592553
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
8079.071573246115
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
9756.151993859587
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
9737.529138917773
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
11793.946617008078


10040.901876777214

In [ ]:
class MacdCross(Strategy):

    id = 0
    take_profit_ratio = 1.02
    stop_loss_ratio = 0.95

    def init(self):
        price = self.data.Close
        self.macd_line = self.I(MACD, price)[0]
        self.signal_line = self.I(MACD, price)[1]

    def next(self):
        entry_price = self.data.Close[-1]

        if crossover(self.macd_line, self.signal_line):
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long {self.id}')
            self.id += 1

        elif crossover(self.signal_line, self.macd_line):
            for trade in self.trades:
                if trade.tag ==f'Long {self.id-1}':
                    trade.close()